# Construct HyDE RAG

In [1]:
import sys
sys.path.append("../..")
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.service.provider import ProviderService

provider = ProviderService()

In [23]:
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

ai = provider.get_simple_gemini_pro()

map_reduce_chain = load_summarize_chain(
    ai,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

ValueError: Counting tokens is not yet supported!

## Test

In [5]:
provider.config.enable_tracing("CODEBOT")

In [3]:
from src.utils.config_utils import get_gemini_hyde_config
text_idx = "text-raptor-medium_index_tdt_vi"
vec_idx = "vec-raptor-medium_index_tdt_vi"

config = get_gemini_hyde_config()
config.vector_index = vec_idx
config.text_index = text_idx

rag = HydeHybridSearchRAG(provider=provider, k=4, config=config)

In [4]:
ques = "Nếu học ngành 2 thì học phí sẽ được tính như thế nào"
ques = "Thời hạn cuối cùng để nộp bảng điểm Tiếng Anh chuẩn đầu ra để được xét tốt nghiệp là khi nào"
ques = "Chuẩn đầu ra Tiếng Anh của Sinh viên"
ques = "Điểm P cho học phần Tiếng Anh cuối có nghĩa là gì"
ques = "Em là sinh viên ở kí túc xá, theo quy định thì sinh viên nội trú phải tham gia câu lạc bộ, nếu em tham gia câu lạc bộ của khoa-ngành của mình thì có được tính không"
ques = "Khi nhận bằng tốt nghiệp cần mang theo giấy tờ gì"
ans = rag.answer(ques)
ans

'Thông tin bạn cung cấp không đề cập đến việc cần mang theo giấy tờ gì khi nhận bằng tốt nghiệp. \n\nTuy nhiên, bạn nên liên hệ trực tiếp với trường Đại học Tôn Đức Thắng để xác nhận chính xác những giấy tờ cần thiết trước khi đến nhận bằng. \n'

In [6]:
rag.retrieve_docs

[[Document(page_content='. Những giấy tờ cần mang theo khi làm Thủ tục nhập học  - Giấy chứng nhận kết quả thi THPT quốc gia năm 2018 : bản chính;  -\xa0Giấy báo nhập học (chỉ xuất trình khi làm thủ tục nhập học, không nộp);  -\xa0Sơ yếu lý lịch sinh viên (tải tại đây) , Giấy xác nhận tìm hiểu thông tin “Những điều cần biết khi học tại TDTU” (tải tại đây) : 01 bản;  -\xa0Giấy khai sinh: 01 bản sao hoặc bản photocopy (có công chứng);  -\xa0Học bạ THPT: 02 bản photocopy (có công chứng);  -\xa0Bằng tốt nghiệp THPT (tốt nghiệp trước năm 2018) hoặc Giấy chứng nhận tốt nghiệp tạm thời (tốt nghiệp năm 2018): 03 bản photocopy (có công chứng);  -\xa0Giấy chứng nhận hưởng chế độ ưu tiên tuyển sinh (nếu thuộc diện ưu tiên): 01 bản photocopy (có công chứng);  -\xa0Giấy chứng minh nhân dân hoặc Căn cước công dân: 02 bản photocopy (có công chứng); -\xa0Hộ khẩu: 01 bản photocopy (có công chứng);  -\xa0Bản photocopy chứng chỉ tiếng Anh quốc tế còn thời hạn (nếu có - nộp lúc xác nhận nhập học);  -\xa0G

## Learn

In [2]:
rag = HydeRAG(provider=provider, index="test-basic_test_tdt_dataset")

In [7]:
provider.config.enable_tracing(project="playground")
resp = rag.embeddings.invoke("Việt nam là quốc gia như thế nào?")

In [8]:
resp

{'QUESTION': 'Việt nam là quốc gia như thế nào?',
 'text': 'Việt Nam là một quốc gia nằm ở Đông Nam Á, với đường bờ biển dài và nhiều cảnh quan thiên nhiên đa dạng, từ đồng bằng trù phú đến những ngọn núi hùng vĩ. Đất nước này có lịch sử lâu đời và phong phú, với nhiều di tích lịch sử và văn hóa. Việt Nam cũng là một quốc gia năng động và đang phát triển, với nền kinh tế phát triển nhanh chóng và một nền văn hóa sôi động. Người dân Việt Nam được biết đến với sự thân thiện, hiếu khách và tinh thần cộng đồng mạnh mẽ.'}

In [9]:
docs = rag.es.similarity_search_with_relevance_scores(query=resp['text'], k=4)
docs

[(Document(page_content='Nhà trường sẽ tổ chức mở lớp ôn tập cho sinh viên được cứu xét gia hạn đào tạo nếu môn học không còn mở hoặc môn học không có tổ chức giảng dạy trong học kỳ hiện tại. Sinh viên khi được cứu xét gia hạn đào tạo phải liên hệ với Văn phòng Khoa để được hướng dẫn đăng ký học cho kịp xét tốt nghiệp theo đúng thời hạn được cứu xét gia hạn.', metadata={'doc_id': '3369b8e3-1b47-11ef-a755-d38426455a06'}),
  0.8390565),
 (Document(page_content='10 điểm;  - Ngành Kiến trúc, Thiết kế đồ họa, Thiết kế nội thất, Thiết kế thời trang, Thiết kế công nghiệp thí sinh phải đạt yêu cầu về năng khiếu do khoa chuyên môn xét;  - Thí sinh người nước ngoài học tập và tốt nghiệp THPT Quốc gia tại Việt Nam phải xếp loại học lực 3 năm THPT từ khá trở lên;  - Thí sinh đăng ký tuyển sinh chương trình đại học bằng tiếng Việt phải đạt trình độ tiếng Việt tương đương B2 (bậc 4/6) trở lên theo Khung năng lực tiếng Việ t theo quy định tại Thông tư số 17/2015/TT-BGDĐT ngày 01/9/2015 của Bộ trưởng 

In [10]:
docs = rag.es.max_marginal_relevance_search(query=resp['text'], k=4)

docs

[Document(page_content='Nhà trường sẽ tổ chức mở lớp ôn tập cho sinh viên được cứu xét gia hạn đào tạo nếu môn học không còn mở hoặc môn học không có tổ chức giảng dạy trong học kỳ hiện tại. Sinh viên khi được cứu xét gia hạn đào tạo phải liên hệ với Văn phòng Khoa để được hướng dẫn đăng ký học cho kịp xét tốt nghiệp theo đúng thời hạn được cứu xét gia hạn.', metadata={'doc_id': '3369b8e3-1b47-11ef-a755-d38426455a06'}),
 Document(page_content='Quốc gia tại Việt Nam phải xếp loại học lực 3 năm THPT từ khá trở lên;  - Thí sinh đăng ký tuyển sinh chương trình đại học bằng tiếng Việt phải đạt trình độ tiếng Việt tương đương B2 (bậc 4/6) trở lên theo Khung năng lực tiếng Việ t theo quy định tại Thông tư số 17/2015/TT-BGDĐT ngày 01/9/2015 của Bộ trưởng Bộ Giáo dục và Đào tạo, riêng ngành Việt Nam học chuyên ngành Việt Ngữ và Văn hóa xã hội Việt Nam thí sinh đạt trình độ từ A1 (bậc 1/6) trở lên; Thí sinh là người nước ngoài tốt nghiệp THPT Quốc gia tại Việt Nam không xét điều kiện năng lực ti